In [1]:
import os
import sys

os.getcwd()  # Check the current working directory
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..\..')))

import DifferentiationInterface.src.dipy.dipy as di

In [2]:
import time
import torch

# Initialize variables and constants
s0_value = 100.0
K_value = 110.0
r_value = 0.05
sigma_value = 0.3
dt_value = 1.0
z_value = 0.5

N = 1000000
seed = 1

# Computation settings
di.setBackend('torch')

# Define variables
s0 = di.variable(s0_value, 'input','s0')
K = di.variable(K_value, 'input','K')
r = di.variable(r_value, 'input','r')
sigma = di.variable(sigma_value, 'input','sigma')
dt = di.variable(dt_value, 'input','dt')
z = di.variable(z_value, 'randomVariableNormal','z')

input_variables = [s0, K, r, sigma, dt, z]

# Record Tape: Standard Monte Carlo

s = s0 * di.exp((r - sigma **2 / 2) * dt + sigma * di.sqrt(dt) * z)
payoff =  di.if_(s > K, s - K, 0)
PV_standard = di.exp(-r * dt) * di.sum(payoff) / N

warmup_iterations = 10
test_iterations = 100

###
### Test performance of graph eval
###


di.seed(seed)
pre_computed_random_variables = z.NewSample(N)# torch.normal(mean=0, std=1, size=(1, N))


#PV_standard.run_performance_test(input_variables, warmup_iterations = warmup_iterations, test_iterations = test_iterations)


In [3]:
import scipy.special
import numpy as np

backend_array = [  'numpy', 'torch',  'tensorflow', 'jax']

for backend in backend_array:
    di.setBackend(backend)

    # Initialize variables and constants
    s0_value = 100.0
    K_value = 110.0
    r_value = 0.05
    sigma_value = 0.3
    dt_value = 1.0
    z_value = 0.5

    N = 1000000
    seed = 1

    # Define variables
    s0 = di.variable(s0_value, 'input','s0')
    K = di.variable(K_value, 'input','K')
    r = di.variable(r_value, 'input','r')
    sigma = di.variable(sigma_value, 'input','sigma')
    dt = di.variable(dt_value, 'input','dt')
    z = di.variable(z_value, 'randomVariableNormal','z')

    input_variables = [s0, K, r, sigma, dt, z]

    # Record Tape: Standard Monte Carlo

    s = s0 * di.exp((r - sigma **2 / 2) * dt + sigma * di.sqrt(dt) * z)
    payoff =  di.if_(s > K, s - K, 0)
    PV_standard = di.exp(-r * dt) * di.sum(payoff) / N

    warmup_iterations = 10
    test_iterations = 100

    ###
    ### Test performance of graph eval
    ###

    di.seed(seed)
    pre_computed_random_variables = z.NewSample(N)# torch.normal(mean=0, std=1, size=(1, N))

    PV_standard.run_performance_test(input_variables, warmup_iterations = warmup_iterations, test_iterations = test_iterations)


Generated Function Code for numpy:
def myfunc(dt, s0, sigma, z, r, K):
    return ((np.exp(((-r) * dt)) * np.sum(np.where(((s0 * np.exp((((r - ((sigma ** 2) / 2)) * dt) + ((sigma * np.sqrt(dt)) * z)))) > K), ((s0 * np.exp((((r - ((sigma ** 2) / 2)) * dt) + ((sigma * np.sqrt(dt)) * z)))) - K), 0))) / 1000000)

Backend              Result       Gradient (1. entry)  mean runtime    variance runtime
nump                 10.023817    0.500207             0.035776        0.000002       
numpy_jit            10.024070    0.500215             0.030848        0.000014       
torch                10.020878    0.499224             0.002319        0.000026       
torch_optimized      10.020878    0.499224             0.001366        0.000017       
tensorflow           9.999600     0.499357             0.021852        0.000062       
tensorflow_optimized 9.999600     0.499357             0.015737        0.000020       
jax                  10.033216    0.499721             0.017412        0.002793